In [1]:
from bs4 import BeautifulSoup
import requests
import sys
import csv
from xml.sax.saxutils import escape


def parsecsv():
    with open('ARCSLiterature.csv', 'r') as thefile:
        data = csv.DictReader(thefile, delimiter=',', quotechar='"')
        datadict = {}
        for row in data:
            if escape(row['Title']) not in datadict:
                datadict[row['Title']] = [row['Author'],
                                                 row['Publication Year'],
                                                 row['Publication Title'],
                                                 row['Issue'],
                                                 row['Volume'],
                                                 row['DOI'],
                                                 row['Abstract Note'],
                                                 row['Manual Tags']
                                          ]

            else:
                continue
        return datadict


keywords = []

for k, v in parsecsv().items():
    keywordlist = v[7].split("; ")
    for x in keywordlist:
        loweredkeyword = escape(x.lower())
        if len(loweredkeyword) > 1:
            if loweredkeyword not in keywords:
                keywords.append(loweredkeyword)
                
print(len(keywords))





5913


In [2]:
def scrapeQ(query):
    #query = "american pika"
    getUrl= 'https://en.wikipedia.org/wiki/' + query
    url = getUrl
    #print(url)
    content = requests.get(url).content
    soup = BeautifulSoup(content,'lxml')
    #extracting page title
    firstHeading = soup.find('h1',{'class' : 'firstHeading'})

    if "disambiguation" in firstHeading.text:
        print("Disambiguation")
    else: 

        QNumber = soup.find('li', {'id' : 't-wikibase'})
        if QNumber is not None:
            #print(firstHeading.text)
            #print(QNumber)
            q = QNumber.a['href'].rsplit('/')[-1]
            #print(q)
            return (firstHeading.text.lower(), q)
        else:
            #print("Nothing found")
            return (firstHeading.text, None)

In [3]:
scrapeQ("american pika")

('american pika', 'Q469908')

In [4]:
counter = 0
nonecounter = 0


with open("keyword_wikiQs.csv", "w") as csvfile:
    fieldnames = ['keyword', 'Q']
    kwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    kwriter.writeheader()

    for k in keywords:
        try:
            wikiQ = scrapeQ(k)
            if wikiQ[1] is not None:
                #print(wikiQ)
                counter += 1
                #print({'keyword': wikiQ[0], 'Q': wikiQ[1]})
                kwriter.writerow({'keyword': wikiQ[0], 'Q': wikiQ[1]})
            else:
                #print(wikiQ[0], "had no Q value.")
                nonecounter += 1
        except TypeError:
            continue
            
print("Scraping done.")
print("number of valid Qs: " + str(counter))
print("number of no Qs: " + str(nonecounter))

Disambiguation
Disambiguation
Disambiguation
Disambiguation
Disambiguation
Disambiguation
Disambiguation
Scraping done.
number of valid Qs: 2986
number of no Qs: 2920
